In [1]:
#@title *Prepare* { display-mode: "form", run: "auto" }

#@markdown # **Prepare the environment**
#@markdown Execute to install necessary packages, modules, fonts

#@markdown Select LeGen version (leave empty for latest):
legen_version = "latest" #@param {type:"string"}

#@markdown Mount Google Drive at /content/drive (your drive folder at /content/drive/MyDrive):
mount_gdrive = True #@param {type:"boolean"}
#@markdown Force mount again. Useful for bug cases:
force_remount = False #@param {type:"boolean"}

# remove the old log
!rm -rf error_log.txt > /dev/null

import os
import subprocess
import shutil

try:
    from google.colab import drive
except ImportError:
    drive = None

# mount Google Drive
if mount_gdrive:
    if drive is None:
        print("Mounting skipped: google.colab is unavailable in this environment.")
    else:
        print("Mounting your Google Drive | Waiting user Allow Access | ", end='')
        try:
            drive.mount('/content/drive/', force_remount=force_remount)
        except Exception as e:
            print(f"[✗]: {e}")

# install uv
print("Installing uv...", end='')
if shutil.which('uv') is None:
    with open('/content/error_log.txt', 'a') as f:
        process = subprocess.Popen('pip3 install uv', shell=True, stderr=f)
        return_code = process.wait()
        print("[✔]" if return_code == 0 else "[✗]")
else:
    print("[✔] (already installed)")

# install legen
print(f"Installing LeGen ({legen_version})...", end='')
install_cmd = 'uv tool install legen'
if legen_version and legen_version != "latest":
    install_cmd = f'uv tool install legen=={legen_version}'

with open('/content/error_log.txt', 'a') as f:
    process = subprocess.Popen(install_cmd, shell=True, stderr=f)
    return_code = process.wait()
    print("[✔]" if return_code == 0 else "[✗]")

# install libcudnn8
print("Install libcudnn8...", end='')
with open('/content/error_log.txt', 'a') as f:
    process = subprocess.Popen('sudo apt install -y libcudnn8', shell=True, stderr=f)
    return_code = process.wait()
    print("[✔]" if return_code == 0 else "[✗]")

# install ffmpeg and xvfb
print("Installing FFmpeg and xvfb...", end='')
with open('/content/error_log.txt', 'a') as f:
    process = subprocess.Popen('apt update -y ; apt install ffmpeg xvfb -y', shell=True, stderr=f)
    return_code = process.wait()
    print("[✔]" if return_code == 0 else "[✗]")

# install pip requirements.txt updating
print("Installing fonts...", end='')
with open('/content/error_log.txt', 'a') as f:
    process = subprocess.Popen('echo ttf-mscorefonts-installer msttcorefonts/accepted-mscorefonts-eula select true | debconf-set-selections && apt install -y ttf-mscorefonts-installer && fc-cache -f -v', shell=True, stderr=f)
    return_code = process.wait()
    print("[✔]" if return_code == 0 else "[✗]")

# create a virtual display
os.system('Xvfb :1 -screen 0 2560x1440x8  &') # create virtual display with size 1600x1200 and 8 bit color. Color can be changed to 24, 16 or 8
os.environ['DISPLAY'] = ':1.0' # tell X clients to use our virtual DISPLAY :1.0.

print("\nPreparation tasks done.")

Mounting your Google Drive | Waiting user Allow Access | Mounted at /content/drive/
Installing uv...[✔] (already installed)
Installing LeGen (latest)...[✔]
Install libcudnn8...[✔]
Installing FFmpeg and xvfb...[✔]
Installing fonts...[✔]

Preparation tasks done.


In [2]:
#@title *Configure* { display-mode: "form", run: "auto" }
#@markdown # **Define Software Settings**
#@markdown ---
#@markdown ## General Options
#@markdown Set where your files are located (your Drive is the base /content/drive/MyDrive):
import os
from urllib.parse import urlparse

overwrite_existing = True #@param {type:"boolean"}
normalize = False #@param {type:"boolean"}
generate_srt = True #@param {type:"boolean"}
generate_txt = True #@param {type:"boolean"}
generate_embed_softsub = False #@param {type:"boolean"}
generate_hardsub = False #@param {type:"boolean"}
copy_extra_files = True #@param {type:"boolean"}
#@markdown Process existing input subtitle files (.srt) found in the input folder (translate/TLTW):
process_input_subs = False #@param {type:"boolean"}

input_path = "/content/drive/MyDrive/2026/Sync Obsidian/Minhas Notas/00_Projetos/Câmara dos Deputados/Estratégia/Curso Regular/Direito Constitucional/Aula 03" #@param {type:"string"}
download_path = "/content/drive/MyDrive/2026/Sync Obsidian/Minhas Notas/00_Projetos/Câmara dos Deputados/Estratégia/Curso Regular/Direito Constitucional/Aula 03" #@param {type:"string"}  # yt-dlp stores media here with subtitles embedded in the MP4
output_softsubs_path = "/content/drive/MyDrive/2026/Sync Obsidian/Minhas Notas/00_Projetos/Câmara dos Deputados/Estratégia/Curso Regular/Direito Constitucional/Aula 03" #@param {type:"string"}
output_hardsubs_path = "/content/drive/MyDrive/2026/Sync Obsidian/Minhas Notas/00_Projetos/Câmara dos Deputados/Estratégia/Curso Regular/Direito Constitucional/Aula 03" #@param {type:"string"}

def _looks_like_url(value: str) -> bool:
    try:
        parsed = urlparse(value)
        return parsed.scheme in ("http", "https") and bool(parsed.netloc)
    except Exception:
        return False

# Create directories helper
#@markdown Click the button below if you need to create the directories above:
if "_legen_create_dirs" not in globals():
    def _legen_create_dirs():
        if input_path and not _looks_like_url(input_path):
            os.makedirs(input_path, exist_ok=True)
        elif _looks_like_url(input_path):
            print("Input is a URL. Skipping local directory creation for it.")
        if download_path:
            os.makedirs(download_path, exist_ok=True)
        if generate_embed_softsub:
            os.makedirs(output_softsubs_path, exist_ok=True)
        if generate_hardsub:
            os.makedirs(output_hardsubs_path, exist_ok=True)
        print("Directories ready.")
try:
    from google.colab import output as _colab_output
    _colab_output.register_callback('legen_create_dirs', _legen_create_dirs)
except Exception:
    pass
#@markdown
#@markdown <button onclick="google.colab.kernel.invokeFunction('legen_create_dirs', [], {});">Create directories</button>

#@markdown ---
#@markdown ## Transcription Settings:
transcription_engine = 'WhisperX' # @param ["Whisper", "WhisperX"]
transcription_device = 'auto' #@param ["auto", "cpu", "cuda"]
transcription_model = 'large-v3-turbo' #@param ["tiny", "small", "medium", "large", "large-v1", "large-v2", "large-v3", "turbo", "large-v3-turbo", "distil-large-v2", "distil-medium.en", "distil-small.en", "distil-large-v3"]
transcription_vad = 'Silero' # @param ["Silero", "Pyannote"]
compute_type = 'default' # @param ["default", "int8", "int16", "float16", "float32"]
batch_size = 12 #@param {type:"number"}
transcription_input_lang = 'pt' #@param ["auto detect", "af", "am", "ar", "as", "az", "ba", "be", "bg", "bn", "bo", "br", "bs", "ca", "cs", "cy", "da", "de", "el", "en", "es", "et", "eu", "fa", "fi", "fo", "fr", "gl", "gu", "ha", "haw", "he", "hi", "hr", "ht", "hu", "hy", "id", "is", "it", "ja", "jw", "ka", "kk", "km", "kn", "ko", "la", "lb", "ln", "lo", "lt", "lv", "mg", "mi", "mk", "ml", "mn", "mr", "ms", "mt", "my", "ne", "nl", "nn", "no", "oc", "pa", "pl", "ps", "pt", "ro", "ru", "sa", "sd", "si", "sk", "sl", "sn", "so", "sq", "sr", "su", "sv", "sw", "ta", "te", "tg", "th", "tk", "tl", "tr", "tt", "uk", "ur", "uz", "vi", "yi", "yo", "zh"]

#@markdown ---
#@markdown ## Translation Settings:
#@markdown Set the destination language code. Set to same as original to skip translation
target_language_code = 'pt-BR' #@param ["af", "sq", "am", "ar", "hy", "as", "ay", "az", "bm", "eu", "be", "bn", "bho", "bs", "bg", "ca", "ceb", "zh-CN", "zh-TW", "co", "hr", "cs", "da", "dv", "doi", "nl", "en", "eo", "et", "ee", "fil", "fi", "fr", "fy", "gl", "ka", "de", "el", "gn", "gu", "ht", "ha", "haw", "he", "hi", "hmn", "hu", "is", "ig", "ilo", "id", "ga", "it", "ja", "jv", "kn", "kk", "km", "rw", "gom", "ko", "kri", "ku", "ckb", "ky", "lo", "la", "lv", "ln", "lt", "lg", "lb", "mk", "mai", "mg", "ms", "ml", "mt", "mi", "mr", "mni-Mtei", "lus", "mn", "my", "ne", "no", "ny", "or", "om", "ps", "fa", "pl", "pt-BR", "pt-PT", "pa", "qu", "ro", "ru", "sm", "sa", "gd", "nso", "sr", "st", "sn", "sd", "si", "sk", "sl", "so", "es", "su", "sw", "sv", "tl", "tg", "ta", "tt", "te", "th", "ti", "ts", "tr", "tk", "ak", "uk", "ur", "ug", "uz", "vi", "cy", "xh", "yi", "yo", "zu"]
#@markdown Select translation engine:
translate_engine = 'google' #@param ["google", "gemini"]
#@markdown If using Gemini (translation) and/or TLTW (summaries), paste your API keys here (separate with commas or new lines):
gemini_api_keys = '' #@param {type:"string"}

#@markdown ---
#@markdown ## TLTW (Too Long To Watch)
#@markdown Generate a structured summary (.md) from subtitles (uses translated subtitles if generated):
enable_tltw = False #@param {type:"boolean"}
#@markdown Optional: output folder for TLTW summaries (leave empty to use softsubs folder):
tltw_output_path = "" #@param {type:"string"}

#@markdown ---
#@markdown ## Video Settings:
codec_video = "h264"  #@param ["h264", "hevc", "mpeg4"]
video_hardware_api = "auto"  #@param ["auto", "none", "nvenc", "vaapi", "amf", "v4l2m2m", "qsv", "videotoolbox", "cuvid"]
codec_audio = "aac"  #@param ["aac", "libopus", "libmp3lame", "pcm_s16le"]

def collect_legen_settings():
    subtitle_formats = []
    if generate_srt:
        subtitle_formats.append("srt")
    if generate_txt:
        subtitle_formats.append("txt")
    if not subtitle_formats:
        subtitle_formats = ["srt"]
    gemini_keys = [
        key.strip()
        for key in gemini_api_keys.replace(',', '\n').splitlines()
        if key.strip()
    ]
    return {
        "input_path": input_path,
        "download_path": download_path,
        "output_softsubs_path": output_softsubs_path,
        "output_hardsubs_path": output_hardsubs_path,
        "overwrite_existing": overwrite_existing,
        "normalize": normalize,
        "copy_extra_files": copy_extra_files,
        "generate_srt": generate_srt,
        "generate_txt": generate_txt,
        "generate_embed_softsub": generate_embed_softsub,
        "generate_hardsub": generate_hardsub,
        "process_input_subs": process_input_subs,
        "transcription_engine": transcription_engine,
        "transcription_device": transcription_device,
        "transcription_model": transcription_model,
        "transcription_vad": transcription_vad,
        "compute_type": compute_type,
        "batch_size": batch_size if transcription_engine == 'WhisperX' else None,
        "transcription_input_lang": transcription_input_lang,
        "target_language_code": target_language_code,
        "translate_engine": translate_engine,
        "gemini_api_keys": gemini_keys,
        "enable_tltw": enable_tltw,
        "tltw_output_path": tltw_output_path,
        "codec_video": codec_video,
        "video_hardware_api": video_hardware_api,
        "codec_audio": codec_audio,
        "subtitle_formats": subtitle_formats,
        "input_is_url": _looks_like_url(input_path),
    }


In [ ]:
#@title *Run* { display-mode: "form" }
#@markdown # **Run LeGen.py**

if "collect_legen_settings" not in globals():
    raise RuntimeError("Run the Configure cell before executing this step.")

print("Starting LeGen...")
import os
import shlex
import torch

settings = collect_legen_settings()
settings["transcription_vad"] = settings["transcription_vad"].lower()

if settings.get("enable_tltw") and not settings.get("gemini_api_keys"):
    raise ValueError("TLTW requires at least one Gemini API key. Paste it in the Configure cell.")

if '_looks_like_url' not in globals():
    from urllib.parse import urlparse
    def _looks_like_url(value: str) -> bool:
        try:
            parsed = urlparse(value)
            return parsed.scheme in ("http", "https") and bool(parsed.netloc)
        except Exception:
            return False

input_is_url = settings.get("input_is_url", False)
if not input_is_url:
    os.makedirs(settings["input_path"], exist_ok=True)
else:
    print("Input path is a URL. The downloader will fetch the media automatically.")
if settings.get("download_path"):
    os.makedirs(settings["download_path"], exist_ok=True)
if settings["generate_embed_softsub"]:
  os.makedirs(settings["output_softsubs_path"], exist_ok=True)
if settings["generate_hardsub"]:
  os.makedirs(settings["output_hardsubs_path"], exist_ok=True)

subtitle_formats_value = ','.join(settings["subtitle_formats"])

# build query
query = f" -i '{settings['input_path']}'"
query += f" --output_softsubs '{settings['output_softsubs_path']}'"
query += f" --output_hardsubs '{settings['output_hardsubs_path']}'"
if settings.get("download_path"):
    query += f" --output_downloads {shlex.quote(settings['download_path'])}"
query += " --process_input_subs" if settings.get("process_input_subs") else ""
query += " --overwrite" if settings["overwrite_existing"] else ""
query += " --norm" if settings["normalize"] else ""
query += " --copy_files" if not settings["copy_extra_files"] else ""
query += f" --subtitle_formats {shlex.quote(subtitle_formats_value)}" if subtitle_formats_value else ""
query += " --disable_softsubs" if not settings["generate_embed_softsub"] else ""
query += " --disable_hardsubs" if not settings["generate_hardsub"] else ""
query += f" -ts:e {settings['transcription_engine'].lower()}"
query += f" -ts:d {settings['transcription_device'].lower()}"
query += f" -ts:m {settings['transcription_model']}"
query += f" -ts:c {settings['compute_type']}"
query += f" -ts:v {settings['transcription_vad']}"
query += f" -ts:b {settings['batch_size']}" if settings["batch_size"] is not None else ""
query += f" --input_lang {settings['transcription_input_lang']}" if settings["transcription_input_lang"] != "auto detect" else ""
query += f" --translate {settings['target_language_code'].lower()}"
query += f" --translate_engine {settings['translate_engine']}"
if settings.get("enable_tltw"):
    query += " --tltw"
    if settings.get("tltw_output_path"):
        query += f" --output_tltw {shlex.quote(settings['tltw_output_path'])}"
if settings.get("translate_engine") == "gemini" or settings.get("enable_tltw"):
    for key in settings["gemini_api_keys"]:
        query += f" --gemini_api_key {shlex.quote(key)}"
query += f" -c:v {settings['codec_video']}" + ("" if settings['video_hardware_api'] == "none" else f"_{settings['video_hardware_api']}" if settings['video_hardware_api'] != "auto" else "_nvenc" if torch.cuda.is_available() else "")
query += f" -c:a {settings['codec_audio']}"

# run python script
print(f"command line: legen {query}", end="\n\n")
!/root/.local/bin/legen $query


Starting LeGen...
command line: legen  -i '/content/drive/MyDrive/2026/Sync Obsidian/Minhas Notas/00_Projetos/Câmara dos Deputados/Estratégia/Curso Regular/Direito Constitucional/Aula 03' --output_softsubs '/content/drive/MyDrive/2026/Sync Obsidian/Minhas Notas/00_Projetos/Câmara dos Deputados/Estratégia/Curso Regular/Direito Constitucional/Aula 03' --output_hardsubs '/content/drive/MyDrive/2026/Sync Obsidian/Minhas Notas/00_Projetos/Câmara dos Deputados/Estratégia/Curso Regular/Direito Constitucional/Aula 03' --output_downloads '/content/drive/MyDrive/2026/Sync Obsidian/Minhas Notas/00_Projetos/Câmara dos Deputados/Estratégia/Curso Regular/Direito Constitucional/Aula 03' --overwrite --subtitle_formats srt,txt --disable_softsubs --disable_hardsubs -ts:e whisperx -ts:d auto -ts:m large-v3-turbo -ts:c default -ts:v silero -ts:b 12 --translate pt-br --translate_engine google -c:v h264_nvenc -c:a aac

/root/.local/share/uv/tools/legen/lib/python3.12/site-packages/pydub/utils.py:300